# Softmax
-------------------------
## notMNIST dataset 
This is a classic case where a softmax regression is a natural, simple model. If you want to assign probabilities to an object being one of several different things, softmax is the thing to do, in addition when we train more sophisticated models, the final step will be a layer of softmax.

## Cross-entropy
We will be using the loss of our model with the following function

$$ H_{y'}(y) = -\sum_i y'_i \log(y_i) $$

Where is our predicted probability distribution, and is the true distribution (the one-hot vector with the digit labels).


### Visualize the dataset...
----------------------------

In [1]:
import numpy as np
import os

In [9]:
path = "./Datasets/notMNIST_small/{}/{}"

A = np.array(os.listdir("./Datasets/notMNIST_small/A/"))
B = np.array(os.listdir("./Datasets/notMNIST_small/B/"))
C = np.array(os.listdir("./Datasets/notMNIST_small/C/"))
D = np.array(os.listdir("./Datasets/notMNIST_small/D/"))
E = np.array(os.listdir("./Datasets/notMNIST_small/E/"))
F = np.array(os.listdir("./Datasets/notMNIST_small/F/"))
G = np.array(os.listdir("./Datasets/notMNIST_small/G/"))
H = np.array(os.listdir("./Datasets/notMNIST_small/H/"))
I = np.array(os.listdir("./Datasets/notMNIST_small/I/"))
J = np.array(os.listdir("./Datasets/notMNIST_small/J/"))

files = np.stack((A, B, C, D, E, F, G, H, I, J), axis=0)
labels = ["A", "B", "C", "D", "E","F", "G", "H", "I", "J"]

## The above ar np arrays holding the names of the files that contains each of the letters in notMNIST
print (path.format("A", A[5]))

./Datasets/notMNIST_small/A/Q2hvY0lDRy5vdGY=.png


In [3]:
## Take a look at the images
from PIL import Image
im = Image.open(path.format("A", A[5]))
im

In [4]:
def getPixels(im):
    pixels = list(im.getdata())
    width, height = im.size
    return np.array([pixels[i * width:(i + 1) * width] for i in range(height)]).reshape(width * height)

getPixels(im).shape

(784,)

In [5]:
def retrieveImages (array, letter):
    return np.array([getPixels(Image.open(path.format(letter, fname))) for fname in array])

retrieveImages(A, "A")

array([[  0,   0,   2, ..., 252, 255, 181],
       [  0,   1,   1, ..., 255, 255, 147],
       [  0,   0,   0, ..., 255, 255, 255],
       ..., 
       [  0,   0,   0, ..., 246, 252, 206],
       [  0,   0,   0, ...,   0,   3,   1],
       [  0,   0,   0, ..., 253, 255, 108]])

In [16]:
# Finally, all is ready to make our trainig matrix!
data = np.stack((retrieveImages(files[i], labels[i]) for i in range(10)), axis=0)
## 10 classes | 1873 examples of each | 784 pixels/dimensions
data.shape

(10, 1873, 784)

In [15]:
classes = np.diag(np.ones(10))
classes

array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]])

## We're now ready to actually make our model!
------------------
A softmax regression has two steps: first we add up the evidence of our input being in certain classes, and then we convert that evidence into probabilities.

To tally up the evidence that a given image is in a particular class, we do a weighted sum of the pixel intensities. The weight is negative if that pixel having a high intensity is evidence against the image being in that class, and positive if it is evidence in favor.

$$ \text{evidence}_i = \sum_j W_{i,~ j} x_j + b_i $$

We then convert the evidence tallies into our predicted probabilities using the "softmax" function:

$$ y_i = \text{softmax}(\text{evidence}_i) = \frac{\exp(x_i)}{\sum_j \exp(x_j)} $$

In [17]:
# Feels good
import tensorflow as tf

In [21]:
x = tf.placeholder(tf.float32, [None, 784])
# x isn't a specific value. It's a placeholder, a value that we'll input when we ask TensorFlow to run a computation. 
# We want to be able to input any number of images, each flattened into a 784-dimensional vector

W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
# Variable. A Variable is a modifiable tensor that lives in TensorFlow's graph of interacting operations. 
# It can be used and even modified by the computation. 
# For machine learning applications, one generally has the model parameters be Variables.

In [22]:
# implement our model
y = tf.nn.softmax(tf.matmul(x, W) + b)

## Train the model
---------------
Recall our loss function will be cross-entropy:
$$ H_{y'}(y) = -\sum_i y'_i \log(y_i) $$
Where y is our predicted probability distribution, and y' is the true distribution (the one-hot vector with the digit labels). In some rough sense, the cross-entropy is measuring how inefficient our predictions are for describing the truth

In [23]:
# placeholder to input the correct answers
y_ = tf.placeholder(tf.float32, [None, 10])
# define loss function
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
# train using the gradient descent algorithm with a learning rate of 0.5.
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [150]:
def getBatch (i, n):
    # i is the letter from which we want the random sample
    # n is the number of samples
    
    # will return n random examples of letters in notMNST and its labels
    xs = data[i][np.random.randint(data[i].shape[0], size=n), :]
    ys = np.repeat([classes[i]], n, axis = 0)
    return xs, ys

def getGeneralBatch (n):
    #will return a batch with batches of n/10 the size for every class
    #long story short, a random batch containing 1/10 intances of every class
    nc = 10
    
    xs, ys = getBatch(0, int(n/nc))
    for i in range(1, nc):
        newXs, newYs = getBatch(i, int(n/nc))
        xs = np.concatenate((xs, newXs))
        ys = np.concatenate((ys, newYs))
        
    return xs, ys
    

In [156]:
xs, ys = getGeneralBatch(100)

In [158]:
xs.shape

(100, 784)

In [159]:
ys.shape

(100, 10)

In [195]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

for _ in range(1000):
    xs, ys = getGeneralBatch(100)
    sess.run(y, feed_dict={x: xs, y_: ys})


In [193]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
ra, fa = getGeneralBatch(1000)
print(sess.run(accuracy, feed_dict={x: ra, y_: fa}))


TypeError: run() missing 1 required positional argument: 'fetches'

In [180]:
mnist.test.labels.shape

(10000, 10)

In [179]:
mnist.test.images.shape

(10000, 784)

In [181]:
data[9].shape

(1873, 784)

In [182]:
np.repeat([classes[9]], 1873, axis = 0).shape

(1873, 10)

In [194]:
ra

array([[  0,   0,   0, ...,   0, 138, 131],
       [  0,   0,   0, ..., 252, 255, 125],
       [  0,   0,   0, ..., 239,  58,   0],
       ..., 
       [  0,   3,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0]])

In [198]:
print(sess.run(y, feed_dict={x: xs}))

SyntaxError: unexpected EOF while parsing (<ipython-input-198-516acf2eebb4>, line 1)

In [199]:
print(sess.run(W))

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
